<a href="https://colab.research.google.com/github/Bombbom/ML4SecurityProject/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM 



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
data = pd.read_pickle("/content/drive/MyDrive/Submit/NT522_Project/dataset_10000_record_fixed_fragment_vectors.pkl")

In [3]:
from __future__ import print_function

import warnings
import numpy as np

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout, LSTM, ReLU, Activation, Bidirectional
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adamax
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight, compute_class_weight

#from parser import parameter_parser
from tensorflow.keras.layers import Layer
#from models.loss_draw import LossHistory
from keras import backend as K
from keras import initializers, regularizers, constraints

In [4]:
vectors = np.stack(data.iloc[:, 0].values)
# vectors = vectors.reshape()
labels = data.iloc[:, 1].values

In [5]:
positive_idxs = np.where(labels == 1)[0]
negative_idxs = np.where(labels == 0)[0]
idxs = np.concatenate([positive_idxs, negative_idxs])
undersampled_negative_idxs = np.random.choice(negative_idxs, len(positive_idxs), replace=True)
resampled_idxs = np.concatenate([positive_idxs, undersampled_negative_idxs])
x_train, x_test, y_train, y_test = train_test_split(vectors[resampled_idxs], labels[resampled_idxs],train_size=0.8,random_state=1, stratify=labels[resampled_idxs])

In [6]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [7]:
dropout = 0.2
# y_train = to_categorical(y_train)
lr = 0.002
batch_size = 64
epochs = 10
threshold = 0.5
  
class_weights =  class_weight.compute_class_weight(class_weight='balanced', classes=[0, 1], y=labels)
adamax = Adamax(lr)
model = Sequential()
model.add(LSTM(300, input_shape=(vectors.shape[1], vectors.shape[2])))
model.add(ReLU())
model.add(Dropout(dropout))
model.add(Dense(300))
model.add(ReLU())
model.add(Dropout(dropout))
model.add(Dense(2, activation='softmax'))

In [8]:
#Tính và lưu trữ kết quả accuracy
model.compile(loss='binary_crossentropy', optimizer=adamax, metrics=['accuracy'])
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/10
125/125 [==============================] - 137s 1s/step - loss: 0.5358 - accuracy: 0.6985
Epoch 2/10
125/125 [==============================] - 124s 997ms/step - loss: 0.4564 - accuracy: 0.7508
Epoch 3/10
125/125 [==============================] - 135s 1s/step - loss: 0.4273 - accuracy: 0.7709
Epoch 4/10
125/125 [==============================] - 135s 1s/step - loss: 0.4033 - accuracy: 0.7856
Epoch 5/10
125/125 [==============================] - 133s 1s/step - loss: 0.3888 - accuracy: 0.7935
Epoch 6/10
125/125 [==============================] - 140s 1s/step - loss: 0.3689 - accuracy: 0.8089
Epoch 7/10
125/125 [==============================] - 144s 1s/step - loss: 0.3534 - accuracy: 0.8190
Epoch 8/10
125/125 [==============================] - 143s 1s/step - loss: 0.3437 - accuracy: 0.8284
Epoch 9/10
125/125 [==============================] - 138s 1s/step - loss: 0.3325 - accuracy: 0.8404
Epoch 10/10
125/125 [==============================] - 155s 1s/step - loss: 0.3217 - acc

In [10]:
_, accuracy_test = model.evaluate(x_test, y_test)
print(accuracy_test)
_, accuracy_train = model.evaluate(x_train, y_train)
print(accuracy_train)
values = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
predictions = (model.predict(x_test, batch_size=batch_size)).round()

tn, fp, fn, tp = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1)).ravel()
FPR=fp / (fp + tn)
print(FPR)
FNR= fn / (fn + tp)
print(FNR)
recall = tp / (tp + fn)
print(recall)
precision = tp / (tp + fp)
print(precision)
F1=(2 * precision * recall) / (precision + recall)
print(F1)
# storeResults("LSTM",(accuracy_train*100),(accuracy_test*100),recall*100,FPR*100,precision*100, F1*100)

63/63 [==============================] - 10s 165ms/step - loss: 0.3735 - accuracy: 0.8320
0.8320000171661377
250/250 [==============================] - 42s 168ms/step - loss: 0.2953 - accuracy: 0.8601
0.8601250052452087
32/32 [==============================] - 9s 273ms/step
0.15
0.186
0.814
0.8443983402489627
0.8289205702647657


In [11]:
print(tn,fp,fn,tp)

850 150 186 814
